# In this notebook, you'll explore how to get data from datastores and using data assets.


You'll need the latest version of the **azureml-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

## Setup

This is important if you want to use your `.py` modules to create jobs, rather than writing your code directly in the notebook.




In [ ]:
import os
import sys

project_root_directory = os.getcwd().split("/notebooks")[0]
sys.path.insert(0, project_root_directory)

## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)




## Configure de env

Let's explore the environments within the workspace.


> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [ ]:
envs = ml_client.environments.list()
for env in envs:
    print(env.name)

Submitting the job with the new custom environment triggers the build of the environment. The first time you use a newly created environment, it can take 10-15 minutes to build the environment, which also means your job will take longer to complete.
You can also choose to manually trigger the build of the environment before you submit a job. The environment only needs to be built the first time you use it.

## List the datastores

When you create the Azure Machine Learning workspace, an Azure Storage Account is created too. The Storage Account includes Blob and file storage and are automatically connected with your workspace as **datastores**. You can list all datastores connected to your workspace:

In [ ]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

Note the `workspaceblobstore` which connects to the **azureml-blobstore-...** container you explored earlier. The `workspacefilestore` connects to the **code-...** file share.

## Create a datastore

Whenever you want to connect another Azure storage service with the Azure Machine Learning workspace, you can create a datastore. Note that creating a datastore, creates the connection between your workspace and the storage, it doesn't create the storage service itself. 

To create a datastore and connect to a (already existing) storage, you'll need to specify:

- The class to indicate with what type of storage service you want to connect. The example below connects to a Blob storage (`AzureBlobDatastore`).
- `name`: The display name of the datastore in the Azure Machine Learning workspace.
- `description`: Optional description to provide more information about the datastore.
- `account_name`: The name of the Azure Storage Account.
- `container_name`: The name of the container to store blobs in the Azure Storage Account.
- `credentials`: Provide the method of authentication and the credentials to authenticate. The example below uses an account key.

**Important**: 
- Replace the **YOUR-STORAGE-ACCOUNT-NAME** with the name of the Storage Account that was automatically created for you. 
- Replace the **XXXX-XXXX** for `account_key` with the account key of your Azure Storage Account. 

Remember you can retrieve the account key by navigating to the [Azure portal](https://portal.azure.com), go to your Storage Account, from the **Access keys** tab, copy the **Key** value for key1 or key2. 

In [ ]:
from azure.ai.ml.entities import AzureBlobDatastore
from azure.ai.ml.entities import AccountKeyConfiguration

In this sample we will create a datastore without storing any credentials. When this datastore is used in a job, the identity used to run the job will also be used to access the datastore. 

In [ ]:
# create a credential-less datastore pointing to a blob container
blob_mobileprice_cleaned = AzureBlobDatastore(
    name="blob_mobileprice_cleaned",
    description="Credential-less datastore pointing to a blob container.",
    account_name="XXXXXX",
    container_name="mlsandbox-container",
    credentials=AccountKeyConfiguration(account_key="XXXXX"),
)
ml_client.create_or_update(blob_mobileprice_cleaned)

List the datastores again to verify that a new datastore named `blob_training_data` has been created:

In [ ]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

## Create data assets

To point to a specific folder or file in a datastore, you can create data assets. There are three types of data assets:

- `URI_FILE` points to a specific file.
- `URI_FOLDER` points to a specific folder.
- `MLTABLE` points to a MLTable file which specifies how to read one or more files within a folder.

You'll create all three types of data assets to experience the differences between them.

To create a `URI_FILE` data asset, you have to specify a path that points to a specific file. The path can be a local path or cloud path.

In the example below, you'll create a data asset by referencing a *local* path. To ensure the data is always available when working with the Azure Machine Learning workspace, local files will automatically be uploaded to the default datastore. In this case, the `diabetes.csv` file will be uploaded to **LocalUpload** folder in the **workspaceblobstore** datastore. 

To create a data asset from a local file, run the following cell:

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = "../data/Mobile-Price-Prediction-cleaned_data.csv"

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="Data asset pointing to a local file, automatically uploaded to the default datastore",
    name="mobile-price-local",
)

ml_client.data.create_or_update(my_data)

To create a `MLTable` data asset, you have to specify a path that points to a folder which contains a MLTable file. The path can be a local path or cloud path. 

In the example below, you'll create a data asset by referencing a *local* path which contains an MLTable and CSV file. 

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = "../data"

my_data = Data(
    path=my_path,
    type=AssetTypes.MLTABLE,
    description="MLTable pointing to diabetes.csv in data folder",
    name="mobile-price-table",
)

ml_client.data.create_or_update(my_data)

To verify that the new data assets have been created, you can list all data assets in the workspace again:

In [ ]:
datasets = ml_client.data.list()
for ds_name in datasets:
    print(ds_name.name)

## Read data in notebook

Initially, you may want to work with data assets in notebooks, to explore the data and experiment with machine learning models. Any `URI_FILE` or `URI_FOLDER` type data assets are read as you would normally read data. For example, to read a CSV file a data asset points to, you can use the pandas function `read_csv()`. 

A `MLTable` type data asset is already *read* by the **MLTable** file, which specifies the schema and how to interpret the data. Since the data is already *read*, you can easily convert a MLTable data asset to a pandas dataframe. 

You'll need to install the `mltable` library (which you did in the terminal). Then, you can convert the data asset to a dataframe and visualize the data.  

In [ ]:
import mltable

registered_data_asset = ml_client.data.get(name="mobile-price-table", version=1)
tbl = mltable.load(f"azureml:/{registered_data_asset.id}")
df = tbl.to_pandas_dataframe()
df.head(5)

## Create an environment for the jobs

If a curated environment doesn't include all the Python packages you need to run your script, you can create your own custom environment. By listing all necessary packages in an environment, you can easily re-run your scripts. All the dependencies are stored in the environment which you can then specify in the job configuration, independent of the compute you use.



In [ ]:
from azure.ai.ml.entities import Environment, BuildContext

In the following example, the local path to the build context folder is specified in the path parameter. 

In [ ]:
env_docker_context = Environment(
    build=BuildContext(path="../"),
    name="docker-context-repo-based-v1",
    description="Environment created from a Docker context.",
)
ml_client.environments.create_or_update(env_docker_context)

In [ ]:
envs = ml_client.environments.list()
for env in envs:
    print(env.name)